In [1]:
import requests
from utils.helper import *
from utils.config import dbInfor
import numpy as np
import pandas as pd
import pymysql
import sqlalchemy
import sys
from datetime import datetime, timedelta

con = get_mysql_connection()
query_ticker = "select distinct ticker from ticker_overview"
list_ticker = get_data_from_mysql(con, query_ticker)

In [10]:
## get date of x session before
def get_day_session_before(con, day, x):
    query = f"SELECT date FROM price_board WHERE ticker = 'FPT' AND date < '{day}' ORDER BY date DESC LIMIT {x}, 1"
    date = get_data_from_mysql(con, query)
    return str(date.iloc[0]['date'].date())

In [8]:
day = str((datetime.today() - timedelta(days=1)).date())

In [90]:
day_65 = get_day_session_before(con, day, 5)
day_130 = get_day_session_before(con, day, 10)
day_260 = get_day_session_before(con, day, 20)

In [91]:
query = f"""
    SELECT ticker,
        max(CASE WHEN date = '{day}' THEN close END) as c,
        max(CASE WHEN date = '{day_65}' THEN close END) AS c_65,
        max(CASE WHEN date = '{day_130}' THEN close END) AS c_130,
        max(CASE WHEN date = '{day_260}' THEN close END) AS c_260
    FROM price_board
    where date = '{day}' or date = '{day_65}' or date = '{day_130}' or date = '{day_260}'
    group by ticker
    """
rsidb = get_data_from_mysql(con, query)

In [93]:
list_ticker = get_data_from_mysql(con,"""select distinct t1.ticker from (price_board as t1 join (select ticker from ticker_overview where comGroupCode = "VNINDEX") as t2 on t1.ticker = t2.ticker)""")

In [94]:
rsidb = rsidb.dropna(subset=['c', 'c_65', 'c_130', 'c_260'])

In [95]:
rsidb['roc_65'] = (rsidb['c'] - rsidb['c_65']) / rsidb['c_65']
rsidb['roc_130'] = (rsidb['c'] - rsidb['c_130']) / rsidb['c_130']
rsidb['roc_260'] = (rsidb['c'] - rsidb['c_260']) / rsidb['c_260']
rsidb['score'] = 0.4 * rsidb['roc_65'] + 0.3 * rsidb['roc_130'] + 0.3 * rsidb['roc_260']

In [96]:
df_all = rsidb.merge(list_ticker, on='ticker')

In [97]:
df_all['rank'] = df_all['score'].rank()

In [98]:
n = len(df_all)

In [99]:
df_all['rank'] = df_all['rank'].apply(lambda x: int((x-1)/n * 100))

In [88]:
df_all[df_all['ticker'] == "CEO"]

,ticker,c,c_65,c_130,c_260,roc_65,roc_130,roc_260,score,rank
35,CEO,22200.0,20200.0,17000.0,15100.0,0.09901,0.305882,0.470199,0.272428,93


In [100]:
df_all.sort_values('rank', ascending=False).head(20)

,ticker,c,c_65,c_130,c_260,roc_65,roc_130,roc_260,score,rank
352,TMT,12300.0,9430.0,9130.0,9130.0,0.304348,0.347207,0.347207,0.330063,99
316,ST8,11900.0,9550.0,8800.0,7790.0,0.246073,0.352273,0.527599,0.362391,99
288,SC5,23000.0,17500.0,17050.0,16600.0,0.314286,0.348974,0.385542,0.346069,99
108,ELC,12650.0,10600.0,10500.0,9000.0,0.193396,0.204762,0.405556,0.260454,99
208,LCG,10250.0,9700.0,8610.0,7050.0,0.056701,0.190476,0.453901,0.215993,98
199,KHG,5270.0,4390.0,4380.0,4050.0,0.200456,0.203196,0.301235,0.231512,98
322,SVI,66600.0,51300.0,54100.0,59200.0,0.298246,0.231054,0.125000,0.226114,98
294,SFI,40700.0,35350.0,33000.0,31500.0,0.151344,0.233333,0.292063,0.218157,98
278,RAL,95200.0,80000.0,79700.0,78000.0,0.190000,0.194479,0.220513,0.200498,97
156,HCD,7300.0,6500.0,6250.0,5400.0,0.123077,0.168000,0.351852,0.205186,97


In [103]:
rsidb['rank'] = rsidb['score'].rank()

In [104]:
n = len(rsidb)

In [105]:
rsidb['rank'] = rsidb['rank'].apply(lambda x: int((x-1)/n * 100))

In [106]:
rsidb

,ticker,c,c_65,c_130,c_260,roc_65,roc_130,roc_260,score,rank
0,A32,31200.0,31200.0,29000.0,30200.0,0.000000,0.075862,0.033113,0.032692,52
1,AAA,8250.0,7400.0,7070.0,6680.0,0.114865,0.166902,0.235030,0.166526,91
2,AAM,11350.0,11700.0,11500.0,11500.0,-0.029915,-0.013043,-0.013043,-0.019792,15
3,AAS,11200.0,11100.0,10500.0,9100.0,0.009009,0.066667,0.230769,0.092834,76
4,AAT,5560.0,5150.0,5300.0,4790.0,0.079612,0.049057,0.160752,0.094787,77
...,...,...,...,...,...,...,...,...,...,...
1613,XPH,8600.0,7500.0,8800.0,8600.0,0.146667,-0.022727,0.000000,0.051848,62
1614,YBC,6500.0,4900.0,3800.0,4400.0,0.326531,0.710526,0.477273,0.486952,99
1615,YBM,5820.0,5500.0,5710.0,5670.0,0.058182,0.019264,0.026455,0.036989,54
1616,YEG,9600.0,8630.0,8620.0,9070.0,0.112399,0.113689,0.058434,0.096596,78


In [107]:
rsidb[rsidb['ticker'] == 'HPG']

,ticker,c,c_65,c_130,c_260,roc_65,roc_130,roc_260,score,rank
600,HPG,21100.0,21650.0,19800.0,17100.0,-0.025404,0.065657,0.233918,0.079711,72


In [46]:
rsidb.sort_values('rank', ascending=False).

,ticker,bsr,c,c_65,c_130,c_260,roc_65,roc_130,roc_260,score,rank
1172,SGH,None,71000.0,27100.0,27700.0,55000.0,1.619926,1.563177,0.290909,1.204196,99
737,LAW,None,24000.0,13900.0,8700.0,8800.0,0.726619,1.758621,1.727273,1.336416,99
549,HEM,None,32900.0,12200.0,11300.0,14000.0,1.696721,1.911504,1.350000,1.657140,99
553,HFB,None,26600.0,9800.0,10600.0,10000.0,1.714286,1.509434,1.660000,1.636544,99
424,EMG,0.0000,31200.0,16500.0,12500.0,12500.0,0.890909,1.496000,1.496000,1.253964,99
354,DNT,None,79300.0,56700.0,24000.0,20000.0,0.398589,2.304167,2.965000,1.740186,99
342,DM7,None,15900.0,9000.0,6500.0,6500.0,0.766667,1.446154,1.446154,1.174359,99
1197,SJC,None,15200.0,5100.0,1500.0,3700.0,1.980392,9.133333,3.108108,4.464589,99
670,INC,None,37900.0,15500.0,14263.0,11779.0,1.445161,1.657225,2.217591,1.740509,99
735,LAI,None,77000.0,35000.0,29200.0,26600.0,1.200000,1.636986,1.894737,1.539517,99
